In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

Using TensorFlow backend.


In [40]:
import os
import math
from keras.utils import np_utils

We will present a few simple yet effective methods that you can use to build a powerful image classifier, using only very few training examples --just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:

1) using the bottleneck features of a pre-trained network
2) fine-tuning the top layers of a pre-trained network

#### In this example, we are going to create our own dataset. Using the GoogleAPI code, we can search any image and download them into training and validation splits of 70 and 30.
#### We will be considering a model of 4 classes of Natural Disaster including Earthquake, Tornado, Hurricane, and Volcanic Eruptions

### Setting the directories of our dataset

In [41]:
# dimensions of our images.
img_width, img_height = 150, 150
os.chdir("/home/ubuntu/modeldata")
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

In [42]:
## Count the number of folders in the directory
def fcount(path):    
    listOfclasses = [name for name in os.listdir(path)] 
    return len(listOfclasses),listOfclasses

In [43]:
## Get the GCD of the two number, this would be use to get the Batch Size during training
def gcd(a, b):
    if(a<b):
        a,b=b,a
    while(b!=0):
        r=b
        b=a%r
        a=r
    return a

## Getting number of training and validation lables

In [101]:
train_labels,nameOfClasses = fcount("/home/ubuntu/modeldata/data/train/")
validation_labels,nameOfClasses= fcount("/home/ubuntu/modeldata/data/validation/")
nameOfClasses.sort()

In [102]:
train_labels,validation_labels

(4, 4)

In [103]:
print(nameOfClasses)

['earthquake', 'hurricane', 'tornado', 'volcanic eruption']


## Preparing the dataset for pre-processing and calculating the GCD

In [78]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = gcd(train_labels*70,validation_labels*30)

In [79]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 280 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


#### Setting the number of Epochs to run and finding the number of Training and Validation samples

In [80]:
epochs = 30
train_samples = train_labels*70
validation_samples = validation_labels*30

## Using the bottleneck features of a pre-trained network: 90% accuracy in a minute

We will use the VGG16 architecture, pre-trained on the ImageNet dataset --a model previously featured on this blog. Because the ImageNet dataset contains several "cat" classes (persian cat, siamese cat...) and many "dog" classes among its total of 1000 classes, this model will already have learned features that are relevant to our classification problem. In fact, it is possible that merely recording the softmax predictions of the model over our data rather than the bottleneck features would be enough to solve our dogs vs. cats classification problem extremely well. However, the method we present here is more likely to generalize well to a broader range of problems, including problems featuring classes absent from ImageNet.

Our strategy will be as follow: we will only instantiate the convolutional part of the model, everything up to the fully-connected layers. We will then run this model on our training and validation data once, recording the output (the "bottleneck features" from th VGG16 model: the last activation maps before the fully-connected layers) in two numpy arrays. Then we will train a small fully-connected model on top of the stored features.

The reason why we are storing the features offline rather than adding our fully-connected model directly on top of a frozen convolutional base and running the whole thing, is computational effiency. Running VGG16 is expensive, especially if you're working on CPU, and we want to only do it once. Note that this prevents us from using data augmentation.

In [48]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

#### We will save the weights of both training and validation weights and then load back to increase the time efficiency

In [49]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, train_samples // batch_size)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [50]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, validation_samples // batch_size)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

#### Creating lables for our training and validation samples, since we would have 4 classes we would needs labels as 0,1,2,3 and so o

In [84]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
b = []
for x in range(len(nameOfClasses)):
    for y in range(1,71):
        b.append(x)
train_labels = np.asarray(b)


validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
c = []
for x in range(len(nameOfClasses)):
    for y in range(1,31):
        c.append(x)
validation_labels = np.asarray(c)

In [86]:
train_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3])

#### Convert 1-dimensional class arrays to n-dimensional class matrices (one hot vector encoding) 

In [87]:
train_labels = np_utils.to_categorical(train_labels,len(nameOfClasses))
validation_labels = np_utils.to_categorical(validation_labels, len(nameOfClasses))

In [88]:
train_labels

array([[ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.]])

#### Creating the model and running it

In [53]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(len(nameOfClasses), activation='softmax'))

model_top.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
model_top.fit(train_data, train_labels,
            epochs=epochs, 
            batch_size=batch_size,
            validation_data=(validation_data, validation_labels))

Train on 280 samples, validate on 120 samples
Epoch 1/30
280/280 [==============================] - 0s - loss: 2.0709 - acc: 0.3071 - val_loss: 1.1080 - val_acc: 0.6583
Epoch 2/30
280/280 [==============================] - 0s - loss: 1.1659 - acc: 0.4893 - val_loss: 1.0181 - val_acc: 0.6083
Epoch 3/30
280/280 [==============================] - 0s - loss: 1.1120 - acc: 0.5107 - val_loss: 1.0329 - val_acc: 0.5500
Epoch 4/30
280/280 [==============================] - 0s - loss: 0.9461 - acc: 0.6714 - val_loss: 0.8272 - val_acc: 0.7833
Epoch 5/30
280/280 [==============================] - 0s - loss: 0.8531 - acc: 0.6964 - val_loss: 0.7265 - val_acc: 0.7833
Epoch 6/30
280/280 [==============================] - 0s - loss: 0.6664 - acc: 0.8036 - val_loss: 0.5793

We reach a validation accuracy of 0.90-0.94: not bad at all.

In [55]:
model_top.save_weights('models/Test_bottleneck_30_epochs.h5')

## Fine-tuning the top layers of a a pre-trained network

To further improve our previous result, we can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:

1) instantiate the convolutional base of VGG16 and load its weights
2) add our previously defined fully-connected model on top, and load its weights
3) freeze the layers of the VGG16 model up to the last convolutional block

Note that:

1) We choose to only fine-tune the last convolutional block rather than the entire network in order to prevent overfitting, since the entire network would have a very large entropic capacity and thus a strong tendency to overfit. The features learned by low-level convolutional blocks are more general, less abstract than those found higher-up, so it is sensible to keep the first few blocks fixed (more general features) and only fine-tune the last one (more specialized features).
2) Fine-tuning should be done with a very slow learning rate, and typically with the SGD optimizer rather than an adaptative learning rate optimizer such as RMSProp. This is to make sure that the magnitude of the updates stays very small

In [89]:
model_2 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [57]:
model_2.output_shape[1:]

(4, 4, 512)

In [59]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_2.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(len(nameOfClasses), activation='softmax'))
top_model.load_weights('models/Test_bottleneck_30_epochs.h5')

In [60]:
#model_vgg.add(top_model)
model = Model(inputs = model_2.input, outputs = top_model(model_2.output))

In [62]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

#### Freeze all convolutional layers up to the last convolutional block

In [63]:
for layer in model.layers[:15]:
    layer.trainable = False

In [64]:
# compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

#### Finally, we start training the whole thing, with a very slow learning rate:

In [65]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size
        )

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size
        )

Found 280 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [67]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

Epoch 1/30
7/7 [==============================] - 10s - loss: 0.2934 - acc: 0.9250 - val_loss: 0.2567 - val_acc: 0.9417
Epoch 2/30
7/7 [==============================] - 8s - loss: 0.2761 - acc: 0.9357 - val_loss: 0.2635 - val_acc: 0.9250

#### This approach gets us to a validation accuracy of 0.95 after 30 epochs. Great success!

In [69]:
model.save_weights('models/FinalStaticModel_30epochs_vgg.h5')

#### Predicting the output

In [167]:
img = load_img("/home/ubuntu/modeldata/data/validation/tornado/86.jpg",False, (img_width, img_height))
x = img_to_array(img)
predictions = model.predict(x.reshape((1,img_width, img_height,3)))
result = dict()
i = 0
for x in predictions.tolist()[0]:
        result[nameOfClasses[i]] = round(x,4)
        i+=1
print(result)

{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}


In [172]:
import glob
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
cwd = os.getcwd()
fname = glob.glob("/home/ubuntu/modeldata/data/validation/volcanic eruption/////*.jpg")
for i in range(len(fname)):    
    img = load_img(fname[i],False, (img_width, img_height))
    x = img_to_array(img)
    prediction = model.predict(x.reshape((1,img_width, img_height,3)),batch_size=32, verbose=0)          
    result = dict()
    i = 0
    for x in prediction.tolist()[0]:
            result[nameOfClasses[i]] = round(x,4)
            i+=1
    print(result)

{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 0.9958, 'tornado': 0.0042, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic eruption': 1.0, 'tornado': 0.0, 'earthquake': 0.0}
{'hurricane': 0.0, 'volcanic erupt